In [1]:
import discord
import random
import os
import asyncio

import pandas as pd
import requests

#########################

import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def has_consecutive_duplicates(lst):
    return any(lst[i] == lst[i+1] for i in range(len(lst)-1))

def get_phase(over):
    if over <= 6:
        return 'P'
    elif over <= 15:
        return 'M'
    else:
        return 'D'
    
def generate_bowling_order(bowler_dict, max_overs=4):
    from collections import Counter, defaultdict
    import random

    # Phase-wise bowler allocation
    phase_bowlers = defaultdict(list)
    for bowler, phases in bowler_dict.items():
        for p in phases:
            phase_bowlers[p].append(bowler)

    # Assigning ranks based on bowler_dict order
    bowler_ranks = {bowler: rank for rank, bowler in enumerate(bowler_dict.keys(), start=1)}
    #print(bowler_ranks)

    def can_bowl(bowler, count, phase, pp_tracker, middle_tracker):
        if count[bowler] >= max_overs:
            return False
        if phase == 'P' and bowler in all_phase_bowlers and pp_tracker[bowler] >= 2:
            return False
        if phase == 'M' and bowler in all_phase_bowlers and middle_tracker[bowler] >= 1:
            return False
        return True

    sequence = [None] * 20
    count = Counter()
    pp_tracker = Counter()
    middle_tracker = Counter()

    all_phase_bowlers = [b for b, phases in bowler_dict.items() if 'P' in phases and 'M' in phases and 'D' in phases]

    top_p_bowlers = [b for b in bowler_dict if 'P' in bowler_dict[b]]
    top_p_bowlers.sort(key=lambda bowler: bowler_dict[bowler].count('P'), reverse=True)

    sequence[0], sequence[1] = top_p_bowlers[0], top_p_bowlers[1]
    count[sequence[0]] += 1
    count[sequence[1]] += 1
    if sequence[0] in all_phase_bowlers:
        pp_tracker[sequence[0]] += 1
    if sequence[1] in all_phase_bowlers:
        pp_tracker[sequence[1]] += 1

    prev_bowler = sequence[1]
    for over in range(2, 20):
        phase = get_phase(over + 1)
        eligible = [b for b in phase_bowlers[phase] if b != prev_bowler and can_bowl(b, count, phase, pp_tracker, middle_tracker)]
        primary = [b for b in eligible if bowler_ranks[b] <= 6]
        backup = [b for b in eligible if bowler_ranks[b] > 6]
        available_bowlers = primary or backup

        if not available_bowlers:
            #print(sequence)
            raise RuntimeError(f"No valid bowlers available for over {over + 1}")

        chosen_bowler = random.choice(available_bowlers)
        sequence[over] = chosen_bowler
        count[chosen_bowler] += 1
        if phase == 'P' and chosen_bowler in all_phase_bowlers:
            pp_tracker[chosen_bowler] += 1
        if phase == 'M' and chosen_bowler in all_phase_bowlers:
            middle_tracker[chosen_bowler] += 1

        prev_bowler = chosen_bowler

    return sequence


def func_1(df):
    # Drop duplicates early on
    df = df.drop_duplicates()

    # Columns to replace NaNs
    df[['wides', 'noballs', 'byes', 'legbyes']] = df[['wides', 'noballs', 'byes', 'legbyes']].fillna(0)
    df[['wicket_type', 'player_dismissed']] = df[['wicket_type', 'player_dismissed']].fillna('')

    # Convert columns to string type where required
    cols_to_string = ['batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler', 'wicket_type', 'player_dismissed']
    df[cols_to_string] = df[cols_to_string].astype(str)

    # Sort once for ease of cumulative calculations
    df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

    # Calculate legality and bowler wicket conditions
    df['legality'] = (df['wides'] + df['noballs']) == 0
    df['bowler_wicket'] = df['wicket_type'].isin(['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket', 'out'])

    # Flagging and categorizing runs
    df['islegal'] = df['legality'].astype(int)
    df['isDotforBatter'] = (df['runs_off_bat'] == 0).astype(int)
    df['isOne'] = (df['runs_off_bat'] == 1).astype(int)
    df['isTwo'] = (df['runs_off_bat'] == 2).astype(int)
    df['isThree'] = (df['runs_off_bat'] == 3).astype(int)
    df['isStrikeRotation'] = df['runs_off_bat'].isin([1, 2, 3, 5, 7]).astype(int)
    df['isFour'] = (df['runs_off_bat'] == 4).astype(int)
    df['isSix'] = (df['runs_off_bat'] == 6).astype(int)
    df['isBowlerWicket'] = df['bowler_wicket'].astype(int)
    df['is_striker_Out'] = (df['striker'] == df['player_dismissed']).astype(int)
    df['is_nonstriker_Out'] = (df['non_striker'] == df['player_dismissed']).astype(int)
    df['isWicket'] = (df['wicket_type'] != '').astype(int)
    df['total_runs'] = df['runs_off_bat'] + df['extras']
    
    # Convert relevant columns to numeric
    df['total_runs'] = pd.to_numeric(df['total_runs'], errors='coerce')
    df['isWicket'] = pd.to_numeric(df['isWicket'], errors='coerce')
    df['islegal'] = pd.to_numeric(df['islegal'], errors='coerce')

    # Fill any NaNs generated due to coercion with 0 (if appropriate for your logic)
    df['total_runs'].fillna(0, inplace=True)
    df['isWicket'].fillna(0, inplace=True)
    df['islegal'].fillna(0, inplace=True)

    # Cumulative sums for runs and wickets
    df['runs_scored'] = df.groupby(['match_id', 'innings'])['total_runs'].cumsum()
    df['wickets_down'] = df.groupby(['match_id', 'innings'])['isWicket'].cumsum()
    df['legal_balls_bowled'] = df.groupby(['match_id', 'innings'])['islegal'].cumsum()
    
    # Calculate run rate and target
    df['run_rate'] = df.apply(lambda x: (6 * x['runs_scored'] / x['legal_balls_bowled']) 
                              if x['legal_balls_bowled'] > 0 else 6 + x['runs_scored'], axis=1)
    

    # Calculate partnership runs with forward filling of last fall of wicket (fow) information
    df['last_fow'] = df['runs_scored'].where(df['isWicket'] == 1).ffill().fillna(0)
    df['partnership_runs'] = df['runs_scored'] - df['last_fow']

    # Calculate cumulative balls faced by each striker and non-striker
    df['isno'] = df['noballs'] > 0
    df['is_faced_by_batter'] = (df['islegal'] | df['isno']).astype(int)

    # Cumulative balls and runs for striker and non-striker
    striker_runs = []
    non_striker_runs = []
    striker_balls = []
    non_striker_balls = []
    
    
    # Calculate last fall of wicket (fow) run rate
    df['last_fow_run_rate'] = df.groupby(['match_id', 'innings'])['run_rate'].apply(lambda x: x.where(df['isWicket'] == 1).ffill())
    df.reset_index(drop=True, inplace=True)

    return df


def func_1(df):
    # Drop duplicates early on
    df = df.drop_duplicates()

    # Columns to replace NaNs
    df[['wides', 'noballs', 'byes', 'legbyes']] = df[['wides', 'noballs', 'byes', 'legbyes']].fillna(0)
    df[['wicket_type', 'player_dismissed']] = df[['wicket_type', 'player_dismissed']].fillna('')

    # Convert columns to string type where required
    cols_to_string = ['batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler', 'wicket_type', 'player_dismissed']
    df[cols_to_string] = df[cols_to_string].astype(str)

    # Sort once for ease of cumulative calculations
    df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

    # Calculate legality and bowler wicket conditions
    df['legality'] = (df['wides'] + df['noballs']) == 0
    df['bowler_wicket'] = df['wicket_type'].isin(['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket', 'out'])

    # Flagging and categorizing runs
    df['islegal'] = df['legality'].astype(int)
    df['isDotforBatter'] = (df['runs_off_bat'] == 0).astype(int)
    df['isOne'] = (df['runs_off_bat'] == 1).astype(int)
    df['isTwo'] = (df['runs_off_bat'] == 2).astype(int)
    df['isThree'] = (df['runs_off_bat'] == 3).astype(int)
    df['isStrikeRotation'] = df['runs_off_bat'].isin([1, 2, 3, 5, 7]).astype(int)
    df['isFour'] = (df['runs_off_bat'] == 4).astype(int)
    df['isSix'] = (df['runs_off_bat'] == 6).astype(int)
    df['isBowlerWicket'] = df['bowler_wicket'].astype(int)
    df['is_striker_Out'] = (df['striker'] == df['player_dismissed']).astype(int)
    df['is_nonstriker_Out'] = (df['non_striker'] == df['player_dismissed']).astype(int)
    df['isWicket'] = (df['wicket_type'] != '').astype(int)
    df['total_runs'] = df['runs_off_bat'] + df['extras']
    
    # Convert relevant columns to numeric
    df['total_runs'] = pd.to_numeric(df['total_runs'], errors='coerce')
    df['isWicket'] = pd.to_numeric(df['isWicket'], errors='coerce')
    df['islegal'] = pd.to_numeric(df['islegal'], errors='coerce')

    # Fill any NaNs generated due to coercion with 0 (if appropriate for your logic)
    df['total_runs'].fillna(0, inplace=True)
    df['isWicket'].fillna(0, inplace=True)
    df['islegal'].fillna(0, inplace=True)

    # Cumulative sums for runs and wickets
    df['runs_scored'] = df.groupby(['match_id', 'innings'])['total_runs'].cumsum()
    df['wickets_down'] = df.groupby(['match_id', 'innings'])['isWicket'].cumsum()
    df['legal_balls_bowled'] = df.groupby(['match_id', 'innings'])['islegal'].cumsum()
    
    # Calculate run rate and target
    df['run_rate'] = df.apply(lambda x: (6 * x['runs_scored'] / x['legal_balls_bowled']) 
                              if x['legal_balls_bowled'] > 0 else 6 + x['runs_scored'], axis=1)
    

    # Calculate partnership runs with forward filling of last fall of wicket (fow) information
    df['last_fow'] = df['runs_scored'].where(df['isWicket'] == 1).ffill().fillna(0)
    df['partnership_runs'] = df['runs_scored'] - df['last_fow']

    # Calculate cumulative balls faced by each striker and non-striker
    df['isno'] = df['noballs'] > 0
    df['is_faced_by_batter'] = (df['islegal'] | df['isno']).astype(int)

    # Cumulative balls and runs for striker and non-striker
    striker_runs = []
    non_striker_runs = []
    striker_balls = []
    non_striker_balls = []
    
    
    # Calculate last fall of wicket (fow) run rate
    df['last_fow_run_rate'] = df.groupby(['match_id', 'innings'])['run_rate'].apply(lambda x: x.where(df['isWicket'] == 1).ffill())
    df.reset_index(drop=True, inplace=True)

    return df


In [3]:
def func_2(df_1):


    #### creation of fours_hit, sixes_hit

    # Sort the DataFrame by 'match_id', 'innings' for forward filling
    df_1.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

    # Group by 'match_id' and 'innings' and forward-fill NaN values within each group
    df_1['fours'] = df_1.groupby(['match_id', 'innings'])['isFour'].cumsum()
    df_1['sixes'] = df_1.groupby(['match_id', 'innings'])['isSix'].cumsum()

    # Reset the DataFrame index
    df_1.reset_index(drop=True, inplace=True)

    # Initialize a dictionary to store player-to-number mappings for each match_id
    player_to_number = {}

    # Initialize a list to store player_number
    player_number = [1]

    # Function to update player-to-number mappings
    def update_player_mapping(player):
        player_to_number[player] = player_number[0]
        player_number[0] += 1

    # Lists to store batting positions for striker and non-striker
    striker_positions = []
    non_striker_positions = []

    # Initialize current match_id
    current_match_id = None

    # Iterate through DataFrame rows
    for index, row in df_1.iterrows():
        #print(index)

        match_id = row['match_id']
        striker = row['striker']
        non_striker = row['non_striker']

        # Check if it's a new match_id
        if current_match_id != match_id:
            current_match_id = match_id
            player_to_number.clear()
            player_number[0] = 1

        # Check if striker has been encountered before in this match
        if striker not in player_to_number:
            update_player_mapping(striker)

        # Check if non-striker has been encountered before in this match
        if non_striker not in player_to_number:
            update_player_mapping(non_striker)

        # Append player numbers to the respective lists
        striker_positions.append(player_to_number[striker])
        non_striker_positions.append(player_to_number[non_striker])

    # Create new columns 'striker_batting_position' and 'non_striker_batting_position'
    df_1['striker_batting_position'] = striker_positions
    df_1['non_striker_batting_position'] = non_striker_positions


    return df_1

In [4]:
#ground stats
df_g = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/ground_stats.csv')

#bowler stats
df_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn1.csv')
df_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn2.csv')

df_bo = pd.concat([df_bo_1,df_bo_2], axis=0)
df_bo.reset_index(drop=True, inplace=True)

#batter stats
df_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn1.csv')
df_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn2.csv')

df_ba = pd.concat([df_ba_1,df_ba_2], axis=0)
df_ba.reset_index(drop=True, inplace=True)

#h2h factor
h2h = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/h2h_stats.csv')

#form factor-batter
form_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn1.csv')
form_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn2.csv')

form_ba = pd.concat([form_ba_1,form_ba_2], axis=0)
form_ba.reset_index(drop=True, inplace=True)

#form factor-bowler
form_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn1.csv')
form_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn2.csv')

form_bo = pd.concat([form_bo_1,form_bo_2], axis=0)
form_bo.reset_index(drop=True, inplace=True)

#spin-pace factor
spin = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/spin_bat.csv')
pace = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/pace_bat.csv')


In [5]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_03')

player_list

In [6]:
def team_map(value):
    if value=='CSK':
        return team_CSK
    elif value=='CSK_B':
        return team_CSK_B
    elif value=='DC':
        return team_DC
    elif value=='DC_B':
        return team_DC_B
    elif value=='GT':
        return team_GT
    elif value=='GT_B':
        return team_GT_B
    elif value=='KKR':
        return team_KKR
    elif value=='KKR_B':
        return team_KKR_B
    elif value=='LSG':
        return team_LSG
    elif value=='LSG_B':
        return team_LSG_B
    elif value=='MI':
        return team_MI
    elif value=='MI_B':
        return team_MI_B
    elif value=='PBKS':
        return team_PBKS
    elif value=='PBKS_B':
        return team_PBKS_B
    elif value=='RCB':
        return team_RCB
    elif value=='RCB_B':
        return team_RCB_B
    elif value=='RR':
        return team_RR
    elif value=='RR_B':
        return team_RR_B
    elif value=='SRH':
        return team_SRH
    elif value=='SRH_B':
        return team_SRH_B
    else:
        return pd.DataFrame()

## run this cell if there's a change to the squad

In [7]:
team_CSK = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='CSK')
team_CSK = team_CSK.sort_values(by='XI').head(11)
team_CSK_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='CSK (2)')
team_CSK_B = team_CSK_B.sort_values(by='XI').head(11)
####

team_DC = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='DC')
team_DC = team_DC.sort_values(by='XI').head(11)
team_DC_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='DC (2)')
team_DC_B = team_DC_B.sort_values(by='XI').head(11)
####

team_GT = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='GT')
team_GT = team_GT.sort_values(by='XI').head(11)
team_GT_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='GT (2)')
team_GT_B = team_GT_B.sort_values(by='XI').head(11)
####

team_KKR = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='KKR')
team_KKR = team_KKR.sort_values(by='XI').head(11)
team_KKR_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='KKR (2)')
team_KKR_B = team_KKR_B.sort_values(by='XI').head(11)

####

team_LSG = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='LSG')
team_LSG = team_LSG.sort_values(by='XI').head(11)
team_LSG_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='LSG (2)')
team_LSG_B = team_LSG_B.sort_values(by='XI').head(11)
####

team_MI = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='MI')
team_MI = team_MI.sort_values(by='XI').head(11)
team_MI_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='MI (2)')
team_MI_B = team_MI_B.sort_values(by='XI').head(11)
####

team_PBKS = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='PBKS')
team_PBKS = team_PBKS.sort_values(by='XI').head(11)
team_PBKS_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='PBKS (2)')
team_PBKS_B = team_PBKS_B.sort_values(by='XI').head(11)
####

team_RCB = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='RCB')
team_RCB = team_RCB.sort_values(by='XI').head(11)
team_RCB_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='RCB (2)')
team_RCB_B = team_RCB_B.sort_values(by='XI').head(11)
####

team_RR = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='RR')
team_RR = team_RR.sort_values(by='XI').head(11)
team_RR_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='RR (2)')
team_RR_B = team_RR_B.sort_values(by='XI').head(11)
####

team_SRH = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='SRH')
team_SRH = team_SRH.sort_values(by='XI').head(11)
team_SRH_B = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx', 
                        sheet_name='SRH (2)')
team_SRH_B = team_SRH_B.sort_values(by='XI').head(11)
####

In [8]:
team_MI_B[['Set','Name','Country','Func_Name','Spin/Pace','XI','bowl','phase','wk','C/VC']]
#team_RCB_B[['Set','Name','Country','Func_Name','Spin/Pace','XI','bowl','phase','wk','C/VC']]

,Set,Name,Country,Func_Name,Spin/Pace,XI,bowl,phase,wk,C/VC
0,R,Prithvi Shaw,India,PP Shaw,0,1.0,NaN,NaN,NaN,NaN
1,UAL7,Akash Vashisht,India,Akash Vashisht,Spin,2.0,6.0,M,NaN,NaN
2,R,Hardik Pandya,India,HH Pandya,Pace,3.0,4.0,"M,D",NaN,NaN
3,UBA3,Gahlaut Rahul Singh,India,Rahul Singh,Pace,4.0,NaN,NaN,NaN,NaN
4,BA4,Ashton Turner,Australia,AJ Turner,Spin,5.0,7.0,M,NaN,NaN
5,R,Rilee Rossouw,South Africa,RR Rossouw,0,6.0,NaN,NaN,1.0,VC
6,R,Jay Gohil,India,Jay Gohil,0,7.0,NaN,NaN,NaN,NaN
7,R,Trent Boult,New Zealand,TA Boult,Pace,8.0,2.0,P,NaN,NaN
8,R,Jasprit Bumrah,India,JJ Bumrah,Pace,9.0,1.0,P,NaN,NaN
9,R,Mohit Sharma,India,MM Sharma,Pace,10.0,5.0,"M,D",NaN,NaN


In [9]:
from contextlib import redirect_stdout

import time as time

t1 = time.time()

import random
from datetime import datetime, timedelta

# Define the teams
teams = ["CSK", "DC", "GT","KKR","LSG","MI","PBKS","RCB","RR","SRH",
        "CSK_B", "DC_B", "GT_B","KKR_B","LSG_B","MI_B","PBKS_B","RCB_B","RR_B","SRH"]


## run this before playoffs

In [10]:
matches_list = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_03 warmup.xlsx',
                            sheet_name='Schedule')

matches_list = matches_list[['Date','Team One','Team Two']]

df_all = pd.DataFrame()

In [11]:
#g = random.choice(range(10,90))
g = 50

ground_factor =  (g/(100-g))   ##(25/75)  #Spin/Pace

df_all= pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/experimentation/df_all_round_sim.csv')

In [12]:
df_all = pd.DataFrame()


df_all = df_all[~(df_all.match_id=='M074')]

for m in range(0,10):

    match_number = m
    with open('run_match_mod.py') as f:
        exec(f.read())

In [13]:
toss_win_team = 'MI'
other_team = 'MI_B'
decision = 'Bowl'

if decision=='Bat':
    batting_team = team_map(toss_win_team)
    bowling_team = team_map(other_team)
else:
    bowling_team = team_map(toss_win_team)
    batting_team = team_map(other_team)


In [14]:
## order of 1st

bowling_order = bowling_team[~(bowling_team.bowl.isna())].Func_Name.values
print(bowling_order)
bowlers_df = bowling_team.dropna(subset=['bowl'])[['bowl','Func_Name','phase']]\
.sort_values('bowl').reset_index(drop=True)
bowlers_df['bowl'] = bowlers_df['bowl'].astype(int)


bowler_dict = dict(zip(bowlers_df['bowl'], bowlers_df['phase'].str.split(',')))
#print(bowler_dict)

if len(bowling_order)<=10:
    fixed = True
    
    attempt = False
    while attempt == False:
        try:
            order = generate_bowling_order(bowler_dict)
            #print(order)
            attempt = True
        except:
            #print("issue")
            continue

        if not has_consecutive_duplicates(order):
                break
    fixed_order_1 = order

['Abhishek Sharma' 'SZ Mulani' 'DAJ Bracewell' 'Umran Malik' 'LH Ferguson'
 'Mohit Avasthi']


In [15]:
## order of 2nd

bowling_team = batting_team
#bowling_team = team_map(other_team)
bowling_order = bowling_team[~(bowling_team.bowl.isna())].Func_Name.values
print(bowling_order)
bowlers_df = bowling_team.dropna(subset=['bowl'])[['bowl','Func_Name','phase']]\
.sort_values('bowl').reset_index(drop=True)
bowlers_df['bowl'] = bowlers_df['bowl'].astype(int)

bowler_dict = dict(zip(bowlers_df['bowl'], bowlers_df['phase'].str.split(',')))

if len(bowling_order)<=10:
    fixed = True
    
    attempt = False
    while attempt == False:
        try:
            order = generate_bowling_order(bowler_dict)
            #print(order)
            attempt = True
        except:
            #print("issue")
            continue

        if not has_consecutive_duplicates(order):
                break
    fixed_order_2 = order

['Akash Vashisht' 'HH Pandya' 'AJ Turner' 'TA Boult' 'JJ Bumrah'
 'MM Sharma' 'SS Ghosh']


fixed_order_2


fixed_order_1, fixed_order_2

In [37]:
df_all = pd.DataFrame()

g = 10

ground_factor =  (g/(100-g))

In [38]:
%%time
from IPython.display import clear_output

m=5
#for m in range(1,3): 
    
num = 0

match_number = m-1
success = False 
clear_output(wait=True)

while not success:  
    try:

        with open('run_match_mod.py') as f:
        #with open('run_match_new.py') as f:
            exec(f.read())
        success = True
        print(f"Match {match_number+1} executed successfully!")

    except Exception as e:
        # Handle the error and retry
        print(f"Error in match {match_number+1}: {e}")
        print("Retrying...")
        num += 1
        print(num)
        clear_output(wait=True)


read
All output has been saved to /Users/roumyadas/Desktop/IPL_Simulation/Season_03/Match_flows/ball-by-ball_5.txt
Match 5 executed successfully!
CPU times: user 15.3 s, sys: 69.5 ms, total: 15.4 s
Wall time: 15.4 s


In [39]:
print("Match Scorecard")
from match_scorecard_warmup import match_scorecard
from IPython.display import clear_output

m_id = 'S03M005'
#for m_id in df_all.match_id.unique():
clear_output(wait=True)
print("--")
match_scorecard(m_id, df_all)
#'M001'

--
All output has been saved to /Users/roumyadas/Desktop/IPL_Simulation/experimentation/Scorecards/matchcard_S03M005.txt


df_all.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/experimentation/df_all_round_sim.csv', index=None)

df_all.match_id.value_counts().reset_index().sort_values(by='index').reset_index(drop=True).tail(5)

df_all[df_all['wicket_type']!='']#.sample(2)

df_all['match_id'].unique()

In [ ]:
#print("-----"*300)

pts_table= pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/points_table.csv')

In [ ]:
print('POINTS TABLE :::')


pts_table['#Won'] = pts_table['points']/2
pts_table['#Won'] = pts_table['#Won'].astype(int)

pts_table['#Lost'] = pts_table['matches_played'] - pts_table['#Won']
pts_table['#Lost'] = pts_table['#Lost'].astype(int)

In [ ]:
new_order = ["team", "matches_played", "points", "#Won","#Lost", "NRR"]
pts_table[new_order]
#.head(5)

pts_table[new_order].to_csv('/Users/roumyadas/Desktop/IPL_Simulation/points_table.csv', index=None)

#batter_stats=pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/batter_stats.csv')
bowler_stats=pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/bowler_stats.csv')


In [ ]:
print('Top BATTERS :::')

bat_reqd = batter_stats#.iloc[15*0:15*2,:]
batter_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/experimentation/batter_stats.csv', index=None)
#######
bat_reqd

In [ ]:
print('Top BOWLERS :::')

bowl_reqd = bowler_stats.drop(['ones','twos','threes','wides','noballs'],axis=1)#.iloc[15*0:15*2,:]
bowler_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/experimentation/bowler_stats.csv', index=None)
#######
bowl_reqd

bat_reqd

batter_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/batter_stats_test.csv', index=None)
bowler_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/bowler_stats_test.csv', index=None)

df_all.columns#['wicket_type'].unique()

batter_stats

df_all.shape

In [ ]:
batter = batter_stats
bowler = bowler_stats
df_sim = df_all

#bat
batter['points'] = batter['runs']*0.5 + \
            np.where(batter['balls_per_dismissal']<1000, batter['balls_per_dismissal']*0.5, batter['balls']*0.5)\
                   + batter['strike_rate']*0.5+\
                   batter['fours']*0.25 + batter['sixes']*0.5 + batter['dot_%']*(-2) + \
                   np.where(batter['bpb']>0, 50/batter['bpb'], 0)

#bowl
bowler['points'] = bowler['wkts']*12.5 + \
            np.where(bowler['strike_rate']<1000, 500/bowler['strike_rate'], 0)\
                   + 100/bowler['economy']+\
                   bowler['fours']*(-0.25) + bowler['sixes']*(-0.5) + bowler['dot_%']*2 + \
                   np.where(((bowler['bpb']>0) & (bowler['bpb']<1000)), bowler['bpb']*5, bowler['balls']*5)

batter['points'] = np.where(batter['balls']>24, batter['points'], 0)
bowler['points'] = np.where(bowler['balls']>24, bowler['points'], 0)


batter_mod = batter.copy()
bowler_mod = bowler.copy()
batter_mod.rename(columns={'striker':'player','points':'points_bat'},inplace=True)
bowler_mod.rename(columns={'bowler':'player','points':'points_ball'},inplace=True)

batter_mod = batter_mod[['player','points_bat']]
bowler_mod = bowler_mod[['player','points_ball']]

player_stats = batter_mod.merge(bowler_mod, on='player',how='outer')
player_stats.fillna(0, inplace=True)
player_stats['total_points'] = player_stats['points_bat']+player_stats['points_ball']

bat_team = df_sim[['striker','batting_team']].drop_duplicates()#.to_dict()
bowl_team = df_sim[['bowler','bowling_team']].drop_duplicates()

bat_team.columns = ['player','team']
bowl_team.columns = ['player','team']

team_info = bat_team.merge(bowl_team, on=['player','team'], how='outer')

#total
player_stats = player_stats.merge(team_info, on='player')
player_stats = player_stats[['player','team','points_bat','points_ball','total_points']]

player_stats = player_stats.sort_values('total_points',ascending=False)
#player_stats.sample(4)

player_stats

batter_stats.to_excel('/Users/roumyadas/Desktop/IPL_Simulation/STATS_S01.xlsx', sheet_name='BAT', index=None)
bowler_stats.to_excel('/Users/roumyadas/Desktop/IPL_Simulation/STATS_S01.xlsx', sheet_name='BOWL', index=None)

## some updates to previous tables

In [ ]:
df_match_info = df_match_info.sort_values('match_id').reset_index(drop=True)

matches_list = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list = matches_list[['Team One','Team Two']].head(len(df_match_info))
df_match_info_mod = pd.concat([df_match_info,matches_list], axis=1)


df_match_info_mod['WL_value'] = ''
df_franchises = {}
for franchise in ['CSK','DC','GT','KKR','LSG','MI','PBKS','RCB','RR','SRH']:
    for index,row in df_match_info_mod.iterrows():
        value = 'NA'
        home = row['Team One']
        away = row['Team Two']
        winner = row['winner']
        
        if (franchise == home): #or (franchise == away)
            if winner == franchise:
                value = 'W'
            else:
                value = 'L'
        elif (franchise == away): #or (franchise == home)
            if winner == franchise:
                value = 'w'
            else:
                value = 'l'
        
        df_match_info_mod.at[index,'WL_value'] = value
        
    df_franchises[franchise] = df_match_info_mod[df_match_info_mod['WL_value']!='NA']['WL_value'].tolist()
    df_franchises[franchise] = ''.join(df_franchises[franchise][::-1])[:5]
               
recent_form = pd.DataFrame(df_franchises.items(), columns=['team', 'Recent Form (Latest first)'])   

pts_table_2 = pts_table.merge(recent_form, on='team', how='left')

pts_table_2 = pts_table_2.round(4)
new_order = ["team", "matches_played", "points", "#Won","#Lost", "NRR","Recent Form (Latest first)"]
pts_table_2 = pts_table_2[new_order]

#run this for playoffs, instead of the upper one
pts_table_2 = pts_table

pts_table_2

### some individual stats

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#######
dots_stats = bowler_stats[['bowler','bowling_team','num_innings','dots','balls','dot_%']]\
                .sort_values(by=['dots','dot_%','balls'], ascending=[False,False,False])
dots_stats = dots_stats[dots_stats['dots']>0]
#######
fours_stats = batter_stats[['striker','batting_team','num_innings','fours','runs','strike_rate']]\
            .sort_values(by=['fours','strike_rate','runs'], ascending=[False,False,False])
fours_stats = fours_stats[fours_stats['fours']>0]
#######
sixes_stats = batter_stats[['striker','batting_team','num_innings','sixes','runs','strike_rate']]\
            .sort_values(by=['sixes','strike_rate','runs'], ascending=[False,False,False])
sixes_stats = sixes_stats[sixes_stats['sixes']>0]
#######
team_runs = df_all.groupby('batting_team')['total_runs'].sum().reset_index()
#######
run_contribution = batter_stats[['striker','batting_team','num_innings','runs','strike_rate','bat_avg']]

run_contribution['run_contribution_%'] = 0.00

for index,row in run_contribution.iterrows():
    batter = row['striker']
    team = row['batting_team']
    runs = row['runs']
    team_run = team_runs[team_runs['batting_team']==team]['total_runs'].unique()[0]
    cont_ = 100*runs/team_run
    run_contribution.at[index,'run_contribution_%'] = np.round(cont_,2)

run_contribution = run_contribution[run_contribution['runs']>0]
run_contribution = run_contribution.sort_values(by='run_contribution_%', ascending=False)
#######
team_wkts = df_all.groupby('bowling_team')['isWicket'].sum().reset_index()
#######
wkt_contribution = bowler_stats[['bowler','bowling_team','num_innings','wkts','balls','strike_rate']]

wkt_contribution['wkt_contribution_%'] = 0.00

for index,row in wkt_contribution.iterrows():
    bowler = row['bowler']
    team = row['bowling_team']
    wkts = row['wkts']
    team_wkt = team_wkts[team_wkts['bowling_team']==team]['isWicket'].unique()[0]
    cont_ = 100*wkts/team_wkt
    wkt_contribution.at[index,'wkt_contribution_%'] = np.round(cont_,2)
    
wkt_contribution = wkt_contribution[wkt_contribution['wkts']>0]
wkt_contribution = wkt_contribution.sort_values(by='wkt_contribution_%', ascending=False)
#######
w3_stats = bowler_stats[['bowler','bowling_team','num_innings','3+_wkts','balls','economy','5+_wkts']]\
                .sort_values(by=['3+_wkts','balls','economy'], ascending=[False,True,True])
w3_stats = w3_stats[w3_stats['3+_wkts']>0]
######
r50_stats = batter_stats[['striker','batting_team','num_innings','50+_scores','runs','strike_rate','100+_scores']]\
                .sort_values(by=['50+_scores','runs','strike_rate'], ascending=[False,False,False])
r50_stats = r50_stats[r50_stats['50+_scores']>0]
######
eco_stats = bowler_stats[['bowler','bowling_team','num_innings','economy','balls','wkts']]\
                .sort_values(by=['economy','balls','wkts'], ascending=[True,False,False])
eco_stats = eco_stats[eco_stats['balls']>0]
######
sr_stats = batter_stats[['striker','batting_team','num_innings','strike_rate','runs','outs']]\
            .sort_values(by=['strike_rate','runs','outs'], ascending=[False,False,True])
sr_stats = sr_stats[sr_stats['runs']>0]
######

dots_stats, fours_stats, sixes_stats, run_contribution, wkt_contribution, w3_stats, r50_stats, eco_stats, sr_stats

In [ ]:
##saving all STATS in one excel##
excel_filename = "/Users/roumyadas/Desktop/IPL_Simulation/experimentation/STATS_S02.xlsx"

# Use ExcelWriter to write multiple sheets
with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    batter_stats.to_excel(writer, sheet_name='BAT', index=False)
    bowler_stats.to_excel(writer, sheet_name='BOWL', index=False)
    pts_table_2.to_excel(writer, sheet_name='POINTS_TABLE', index=False)
    player_stats.to_excel(writer, sheet_name='MVP_points', index=False)
    #---------#
    dots_stats.to_excel(writer, sheet_name='Most_Dots', index=False)
    fours_stats.to_excel(writer, sheet_name='Most_Fours', index=False)
    sixes_stats.to_excel(writer, sheet_name='Most_Sixes', index=False)
    run_contribution.to_excel(writer, sheet_name='Most_Run_Contribution', index=False)
    wkt_contribution.to_excel(writer, sheet_name='Most_Wkt_Contribution', index=False)
    #---------#
    w3_stats.to_excel(writer, sheet_name='Most_3wkt_hauls', index=False)
    r50_stats.to_excel(writer, sheet_name='Most_50+_scores', index=False)
    eco_stats.to_excel(writer, sheet_name='Lowest_Economy', index=False)
    sr_stats.to_excel(writer, sheet_name='Best_Strike_Rate', index=False)
    
    
print(f"Excel file '{excel_filename}' created successfully with 13 sheets!")

bowler_stats

## update the database

In [ ]:
import nbformat
from nbconvert import PythonExporter
from subprocess import run

In [ ]:
import warnings
warnings.filterwarnings('ignore')

with open("IPL Simulation - update on best performances column.ipynb") as f:
    notebook_content = nbformat.read(f, as_version=4)

# Convert to Python script
exporter = PythonExporter()
script, _ = exporter.from_notebook_node(notebook_content)

# Save and execute the script
with open("IPL Simulation - update on best performances column.py", "w") as f:
    f.write(script)

run(["python", "IPL Simulation - update on best performances column.py"])  # Runs the script

In [ ]:
#######################

import nbformat

"""
with open("IPL_Simulation - post-game SIM-stats updation.ipynb", "r") as f:
    notebook = nbformat.read(f, as_version=4)

# Assign unique IDs to all cells
for cell in notebook["cells"]:
    if "id" not in cell or cell["id"] is None:
        cell["id"] = str(uuid.uuid4())  # Generate a new unique ID


# Save the fixed notebook
with open("IPL_Simulation - post-game SIM-stats updation_fixed.ipynb", "w") as f:
    nbformat.write(notebook, f)

print("Fixed notebook saved as 'IPL_Simulation - post-game SIM-stats updation_fixed.ipynb'")

"""
with open("IPL_Simulation - post-game SIM-stats updation_fixed.ipynb") as f:
    notebook_content = nbformat.read(f, as_version=4)

# Convert to Python script
exporter = PythonExporter()
script, _ = exporter.from_notebook_node(notebook_content)

# Save and execute the script
with open("IPL_Simulation - post-game SIM-stats updation_fixed.py", "w") as f:
    f.write(script)

run(["python", "IPL_Simulation - post-game SIM-stats updation_fixed.py"])  # Runs the script

#######################

print("DONE updation")

In [ ]:
##Experimentation area

df_all.columns

batter_stats.columns, bowler_stats.columns

file_path = "/Users/roumyadas/Desktop/IPL_Simulation/STATS_S01.xlsx" 
xls = pd.ExcelFile(file_path)

print(xls.sheet_names)

with open('IPL_Simulation - sim H2H, phase-wise stats updation.ipynb') as f:
    exec(f.read())



In [ ]:
##Experimentation area

batter_stats

df_all['match_id'].value_counts()

In [ ]:
tm = 'SRH'

print(f"{tm} stats!!!")

In [ ]:
batter_stats[batter_stats.batting_team==tm]

In [ ]:
bowler_stats[bowler_stats.bowling_team==tm]

## score-worm plotting

In [ ]:
m_id = 'S02M014'
live_score = df_all[df_all.match_id==m_id][df_all.innings==1].reset_index(drop=True)
df_other = df_all[df_all.match_id==m_id][df_all.innings==2].reset_index(drop=True)

bat_first = live_score['batting_team'].unique()[0]
bat_second = df_other['batting_team'].unique()[0]

print('FIRST INNING SCORE:')
print('RUNS:', live_score['runs_scored'].max())
print('WICKETS:', live_score['wickets_down'].max())
print('BALLS BOWLED:', live_score['legal_balls_bowled'].max())

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
# Assuming you have already imported your dataframe as 'live_score'

# Step 1: Remove duplicate rows based on 'predicted_runs' and 'legal_balls_bowled'
#live_score = live_score[live_score.legal_balls_bowled>=18]
live_score_unique = live_score.drop_duplicates(subset=['runs_scored', 'legal_balls_bowled'], keep='first')
df_other_unique = df_other.drop_duplicates(subset=['runs_scored', 'legal_balls_bowled'], keep='first')


#live_score_unique = live_score_unique[live_score_unique.legal_balls_bowled<=108]

# Calculate the minimum and maximum values for y-axis
min_y_limit = min(100, live_score_unique['runs_scored'].min() * 0.8)
max_y_limit = max(live_score_unique['runs_scored'].max() * 1.05, 200)

# Calculate the x-axis values by dividing 'legal_balls_bowled' by 6
x_values = live_score_unique['legal_balls_bowled'] / 6

# Calculate the x-axis values for the second line
x_values_other = df_other_unique['legal_balls_bowled'] / 6

# Plot the visualization
plt.figure(figsize=(20, 12))  # Set plot size to (15, 9)

# Add grey shade to background
plt.fill_between(x_values, 0, 250, color='lightgrey')

plt.plot(x_values, live_score_unique['runs_scored'], marker='o', linestyle='-', color='b', 
         label=bat_first)
# Plot the second line (df_other_unique)
plt.plot(x_values_other, df_other_unique['runs_scored'], marker='s', linestyle='--', color='orange', 
         label=bat_second)


# Add red dot for wicket
wicket_indices = live_score_unique[live_score_unique['isWicket'] == 1].index
plt.scatter(x_values[wicket_indices], 
            live_score_unique.loc[wicket_indices, 'runs_scored'], 
            color='r', s=250, marker='o', edgecolors='black', label=str(bat_first)+' wickets')

# Adding wicket markers for the second line
wicket_indices_other = df_other_unique[df_other_unique['isWicket'] == 1].index
plt.scatter(x_values_other[wicket_indices_other], 
            df_other_unique.loc[wicket_indices_other, 'runs_scored'], 
            color='purple', s=250, marker='o', edgecolors='black', label=str(bat_second)+' wickets')

# Annotate the last data point of the second line
last_index_other = df_other_unique.index[-1]
last_x_other = x_values_other.iloc[-1]
last_y_other = np.round(df_other_unique['runs_scored'].iloc[-1], 2)
plt.annotate(f'{last_y_other}', xy=(last_x_other, last_y_other), xytext=(last_x_other, last_y_other + 5), fontsize=18,
             arrowprops=dict(facecolor='black', shrink=0.05), xycoords='data', ha='center')


plt.ylim(min_y_limit, max_y_limit)  # Set y-axis range
plt.title('Runs vs Overs Bowled', fontsize=15)  # Updated title
plt.xlabel('Overs Bowled',  fontsize=15)  # Updated x-axis label
plt.ylabel('Runs', va='bottom',  fontsize=15)  # Updated y-axis label
plt.grid(True)

# Set x-axis ticks to display only integer values
plt.xticks(x_values[x_values.apply(lambda x: x.is_integer())])

# Annotate the last data point with its y-axis value just above the end of the line
last_index = live_score_unique.index[-1]
last_x = x_values.iloc[-1]
last_y = np.round(live_score_unique['runs_scored'].iloc[-1], 2)
plt.annotate(f'{last_y}', xy=(last_x, last_y), xytext=(last_x, last_y + 5), fontsize=18,
             arrowprops=dict(facecolor='black', shrink=0.05), xycoords='data', ha='center')

# Plot reference lines for highest and lowest predicted runs achieved
plt.axhline(y=live_score_unique['runs_scored'].max(), color='g', linestyle='--', label='Max Runs')

# Annotate the values of the reference lines on the y-axis
plt.text(2.8, live_score_unique['runs_scored'].max(), f'{live_score_unique["runs_scored"].max():.2f}',  va='bottom', ha='right', fontsize=18)#
#plt.text(2.8, live_score_unique['runs_scored'].min(), f'{live_score_unique["runs_scored"].min():.2f}',  va='top', ha='right', fontsize=18) #

plt.legend()  # Show legend

# Adjust the position of y-label to the bottom
plt.gca().yaxis.set_label_coords( -0.04, 0.15)

plt.show()

print('SECOND INNING SCORE:')
print('RUNS:', df_other['runs_scored'].max())
print('WICKETS:', df_other['wickets_down'].max())
print('BALLS BOWLED:', df_other['legal_balls_bowled'].max())